# **1. Problem Statement**
Dataset is originated from the National Institute of Diabetes and Digestive and Kidney Diseases.
The datasets consists of several medical predictor variables and one target variable, Outcome. Predictor variables includes the number of pregnancies the patient has had, their BMI, insulin level, age, and so on..

The objective of the dataset is to diagnostically predict whether or not a patient has diabetes, based on certain diagnostic measurements.

# **2. Approach**
Use AI/ML to choose and build a Machine Learning model that accurately predicts whether or not the patients in the dataset have diabetes or not.

**Import Python Libraries**

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import zscore
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV, KFold, RepeatedStratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, plot_precision_recall_curve, plot_roc_curve
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier

In [ ]:
## Load pima data into data frame for manipulation using Pandas
diabetic_df= pd.read_csv('../input/pima-indians-diabetes-database/diabetes.csv')
diabetic_df.head()

In [ ]:
## Checking the shape of dataset
diabetic_df.shape

In [ ]:
## Checking column information
diabetic_df.info()

In [ ]:
## Checking for NULL's to handle
diabetic_df.isnull().sum()

Inference: No Null's are there to handle this time

# **3. Exploratory Data Analysis**

In [ ]:
## Using Pair plot
sns.pairplot(diabetic_df, diag_kind='kde', hue='Outcome')

In [ ]:
## Checking 
diabetic_df.describe()

**Inference**: Minimum values for BP and BMI are showing zero which in general it shouldn't be. So there is a need to replace zeroes

In [ ]:
## Classifying patients using Violin plot on Glucose
sns.violinplot(data= diabetic_df, x= "Outcome", y= "Glucose")

In [ ]:
## Classifying patients using Violin plot on BP
sns.violinplot(data=diabetic_df, x='Outcome', y='BloodPressure')

 If we have observed minutely, we can see that the box plot for 1 (Diabetic) inside the violin is a little more away from the horizontal axis than the box plot for 0 (Non Diabetic). It can be implied that diabetics seems to have a higher blood pressure than the non-diabetics. The bottom tail of the violins indicates the zero values we need to replace. We will replace zeros with median values

In [ ]:
from scipy.stats import shapiro
stat, p = shapiro(diabetic_df['BloodPressure'])
print('Statistics=%.3f, p-value=%.3f' % (stat, p))

**Inference**: The p-value is less than 0.05, implying that at 5% Level of Significance (LOS) we reject our null hypothesis (Ho). Therefore, BP does not follow a normal distribution and minimum values have to replaced with median values

In [ ]:
diabetic_df['BloodPressure']=diabetic_df['BloodPressure'].replace(0,diabetic_df['BloodPressure'].median())
diabetic_df['BloodPressure'].describe()

In [ ]:
sns.distplot(diabetic_df['BloodPressure'])

In [ ]:
sns.violinplot(data=diabetic_df, x='Outcome', y='BMI')

In [ ]:
sns.violinplot(data=diabetic_df, x='Outcome', y='Glucose')

In [ ]:
stats, pval = shapiro(diabetic_df['BMI'])
print('Statistics=%.3f, p-value=%.3f' % (stats, pval))

In [ ]:
diabetic_df['BMI']=diabetic_df['BMI'].replace(0,diabetic_df['BMI'].median())
diabetic_df['BMI'].describe()

In [ ]:
sns.distplot(diabetic_df['BMI'])

In [ ]:
stats, pval = shapiro(diabetic_df['Glucose'])
print('Statistics=%.3f, p-value=%.3f' % (stats, pval))

In [ ]:
diabetic_df['Glucose']=diabetic_df['Glucose'].replace(0,diabetic_df['Glucose'].median())
diabetic_df['Glucose'].describe()

In [ ]:
sns.distplot(diabetic_df['Glucose'])

In [ ]:
## Checking pregnant's uniqueness
diabetic_df['Pregnancies'].unique()

In [ ]:
## Plotting bar plot for Pregnants having BP
sns.barplot(data=diabetic_df, y='BloodPressure', x='Pregnancies')

In [ ]:
## Plotting bar plot for Pregnants having Diabetes
sns.barplot(data=diabetic_df, y='Glucose', x='Pregnancies')

In [ ]:
## Checking Data Distribution for Class Variable (0/1)
f, ax = plt.subplots(1,1, figsize=(6,4))
sns.countplot(x="Outcome", data=diabetic_df, hue='Outcome')
total = float(len(diabetic_df))
for p in ax.patches:
    height = p.get_height()
    ax.text(p.get_x()+p.get_width()/2.,
            height + 3,
            '{:1.2f}%'.format(100*height/total),
            ha="center") 
plt.show()

In [ ]:
## Checking Correlation of features using Heatmap
corr= diabetic_df.corr()
fig = plt.figure(figsize=(10,6))
sns.heatmap(corr,  annot=True,cmap ='RdYlGn',linewidths=1.5)

**Data Wrangling** (Handling Outliers)

In [ ]:
## Checking outliers using Box plot
pos=1
plt.figure(figsize=(15,10))
for i in diabetic_df.drop(['Outcome'], axis=1).columns:
  plt.subplot(3,3,pos)
  sns.boxplot(diabetic_df[i],color="red")
  pos+=1

Inference: All columns except Class Variable have outliers. Hence need to eliminate outliers

**IQR Method**:

In [ ]:
## Detect outliers using IQR and Handling outliers
Q1= diabetic_df.quantile(0.25)
Q3= diabetic_df.quantile(0.75)
IQR= Q3-Q1
print(IQR)

In [ ]:
boolean_out= (diabetic_df < (Q1 - 1.5 * IQR)) | (diabetic_df > (Q3 + 1.5 * IQR))
print(boolean_out)

In [ ]:
diabetic_data= diabetic_df[~boolean_out.any(axis=1)]
print('Shape of outliers dataset:', diabetic_df.shape)
print('\n Shape of non outliers dataset:', diabetic_data.shape)

In [ ]:
pos=1
plt.figure(figsize= (15,10))
for i in diabetic_data.drop(['Outcome'], axis=1).columns:
  plt.subplot(3,3, pos)
  sns.boxplot(diabetic_data[i], color='orange')
  pos+=1

Inference: Still have outliers. Let's go with z-score method to remove remaining outliers

#### Z-score Method

In [ ]:
z = np.abs(zscore(diabetic_data))
print(z)

In [ ]:
threshold = 2
print(np.where(z > 2))

In [ ]:
z[17][1]

In [ ]:
pima_data= diabetic_data[(z < 2). all(axis=1)]

In [ ]:
print('Shape of outliers dataset:', diabetic_data.shape)
print('\n Shape of non-outliers dataset:', pima_data.shape)

In [ ]:
pos=1
plt.figure(figsize= (15,10))
for i in pima_data.drop(['Outcome'], axis=1).columns:
  plt.subplot(3,3, pos)
  sns.boxplot(pima_data[i], color='green')
  pos+=1

# **4. Data Preprocessing**

In [ ]:
pima_data.describe()

In [ ]:
pima_data.head()

In [ ]:
pima_data.hist(bins=8,figsize=(8,8))

# **5. Feature Engineering**

In [ ]:
X= pima_data.drop(["Outcome"], axis= 1)
y= pima_data["Outcome"].values

**Split Data**

Train(80%), Test(20%)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)

In [ ]:
Xtrain_scaled= MinMaxScaler(X_train)
ytrain_scaled= MinMaxScaler(y_train)

# **6. Model Selection**
Logistic Regression is performing well in terms of high mean and less standard deviation. Hence choosing the model.

In [ ]:
pipelines = []
pipelines.append(('ScaledLR', Pipeline([('Scaler', StandardScaler()),('LR',LogisticRegression())])))
pipelines.append(('ScaledNB', Pipeline([('Scaler', StandardScaler()),('NB',GaussianNB())])))
pipelines.append(('ScaledKNN', Pipeline([('Scaler', StandardScaler()),('KNN', KNeighborsClassifier())])))
pipelines.append(('ScaledCART', Pipeline([('Scaler', StandardScaler()),('CART', DecisionTreeClassifier())])))
pipelines.append(('ScaledGBM', Pipeline([('Scaler', StandardScaler()),('GBM', GradientBoostingClassifier())])))
pipelines.append(('ScaledRF', Pipeline([('Scaler', StandardScaler()),('RF', RandomForestClassifier())])))
pipelines.append(('ScaledAda', Pipeline([('Scaler', StandardScaler()),('Ada', AdaBoostClassifier())])))
pipelines.append(('ScaledET', Pipeline([('Scaler', StandardScaler()),('ET', ExtraTreesClassifier())])))
pipelines.append(('ScaledXGB', Pipeline([('Scaler', StandardScaler()),('XGB', XGBClassifier())])))
pipelines.append(('ScaledMLP', Pipeline([('Scaler', StandardScaler()),('MLP', MLPClassifier())])))

In [ ]:
results = []
names = []
for name, model in pipelines:
    kfold = KFold(n_splits=10, random_state=21)
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring='accuracy')
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

In [ ]:
# Algorithm comparison
fig = plt.figure(figsize=(18,5))
fig.suptitle('Model Selection by comparision')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.show()

# **Logistic Regression**

In [ ]:
lr_model = LogisticRegression()
lr_model.fit(X_train, y_train)

In [ ]:
lr_preds= lr_model.predict(X_test)

# **Multi Layered Perceptron/Neural Network**

In [ ]:
mlp_model= MLPClassifier(solver='sgd')
mlp_model.fit(X_train, y_train)

In [ ]:
mlp_preds= mlp_model.predict(X_test)

# **7. Model Performance**

In [ ]:
print('Classification report of Logistic Regression model: \n')
print(classification_report(y_test, lr_preds))

In [ ]:
print('Classification report of Neural Network model: \n')
print(classification_report(y_test, mlp_preds))

In [ ]:
## Confusion Matrix of Logistic Regression
confusion_matrix = pd.crosstab(y_test, lr_preds, rownames=['Actual'], colnames=['Predicted'])
sns.heatmap(confusion_matrix, annot=True)
plt.show()

In [ ]:
plot_roc_curve(lr_model, X_test, y_test)
plt.show()

In [ ]:
#Accuracy Score
from sklearn.metrics import accuracy_score
print('Accuracy Score for Logistic Regression:',accuracy_score(lr_preds,y_test))

## **Hyper-Parameter Tuning**

In [ ]:
# Define models and parameters
model = LogisticRegression()
solvers = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
penalty = ['l1','l2','elasticnet']
c_values = [ 1.0, 0.1, 0.01,0.001,1e-5]
mul_class= ['auto','ovr','multinomial']

# define grid search
grid = dict(solver=solvers,penalty=penalty,C=c_values, multi_class=mul_class)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(X_train, y_train)

In [ ]:
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

In [ ]:
print(len(y_test))
print(len(lr_preds))

In [ ]:
diab_predict= pd.DataFrame(y_test, columns=['Actual Class'])
diab_predict["Predicted Class"]= lr_preds
diab_predict.insert(0, 'Modelname', 'Logistic Regression')
diab_predict.head()

In [ ]:
diab_predict.to_csv('/kaggle/working/Submission_LR.csv')